# SETUP

In [ ]:
!nvidia-smi # I want TESLA A100 please 

Thu Aug 11 08:06:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install gdown -q
!pip install efficientnet_pytorch -q
!pip uninstall albumentations --y
!pip install albumentations==1.0.3 -q 
!pip install timm -q
!pip install yacs==0.1.8 pyyaml==5.4.1 -q

In [ ]:
%%capture
!pip uninstall opencv-python --y
!pip install --upgrade opencv-python
!pip install --upgrade opencv-contrib-python
!pip install tensorboardX -q 

In [ ]:
!pip install timm -q
!pip install yacs==0.1.8 pyyaml==5.4.1 -q

In [ ]:
!cp '/content/drive/MyDrive/Microsoft_Rice_Disease/Images.zip' .
!cp '/content/drive/MyDrive/Microsoft_Rice_Disease/Train.csv' .
!cp '/content/drive/MyDrive/Microsoft_Rice_Disease/Test.csv' .
!cp '/content/drive/MyDrive/Microsoft_Rice_Disease/SampleSubmission.csv' .

In [ ]:
# Unzipping the provided images
!mkdir -p images
!unzip -q Images.zip -d images

In [ ]:
exit(0)

# IMPORTS

In [ ]:
import pandas as pd 
import numpy  as np
import random
import os
import cv2
import io
from tqdm import tqdm_notebook as tqdm 
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score ,roc_curve, auc , f1_score , precision_score , log_loss
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import torchvision
import torchvision.models as models
from   torchvision import datasets, transforms, models
from   torchvision import transforms

import timm
import torch
import torch.nn as nn
from   torch.utils.data import DataLoader, Dataset
from   torch.optim.lr_scheduler import MultiStepLR
from   torch.optim.lr_scheduler import OneCycleLR
from   torch.nn import functional as F
from   torch.autograd import Variable
from torch.optim import lr_scheduler

import albumentations
import albumentations as A
import albumentations.augmentations.transforms as AT
from   albumentations.pytorch.transforms import ToTensorV2
import albumentations.augmentations.transforms as AT
import albumentations.augmentations.geometric.transforms as AG
import albumentations.augmentations.geometric.rotate as AGR
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,RandomCrop,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomContrast, RandomBrightness, Flip, OneOf, Compose, RandomGamma, ElasticTransform, ChannelShuffle,RGBShift, Rotate
)

from efficientnet_pytorch import EfficientNet
from scipy.stats import gmean

import yaml
from yacs.config import CfgNode as CN
from functools import reduce , wraps

import warnings
warnings.simplefilter('ignore')

# Data Preparation

In [ ]:
Train = pd.read_csv('Train.csv')
Train.head()

,Image_id,Label
0,id_004wknd7qd.jpg,blast
1,id_004wknd7qd_rgn.jpg,blast
2,id_005sitfgr2.jpg,brown
3,id_005sitfgr2_rgn.jpg,brown
4,id_00stp9t6m6.jpg,blast


In [ ]:
targets = Train.Label.unique().tolist()

Target_Mapper = dict(zip(targets,[i for i in range(len(targets))]))
InverseTarget_Mapper = dict(zip([i for i in range(len(targets))],targets))

In [ ]:
Train.Label = Train.Label.map(Target_Mapper)

In [ ]:
Test = pd.read_csv('Test.csv')

In [ ]:
Train = Train[~Train.Image_id.str.contains('_rgn')].reset_index(drop=True) # Just the RGB images

# Config

In [ ]:
_C = CN()

_C.data = CN()
_C.data.data_path = 'images' # path of the folder that contains train and test images
_C.data.Image_ID_col = 'Image_id'
_C.data.LABELS = 'Label'

_C.preprocess=CN()
_C.preprocess.input_shape = [256,256]
_C.preprocess.crop_shape  = [256,256]

_C.model = CN()
_C.model.name_model = "swinv2_large_window12to16_192to256_22kft1k"

_C.model.train_bs = 64
_C.model.n_accumulate = 8
_C.model.test_bs = 64

_C.model.base_lr = 5e-5
_C.model.weight_decay = 1e-6
_C.model.epochs = 50
_C.model.scheduler = "ReduceLROnPlateau"
_C.model.min_lr = 1e-6
_C.model.factor = 0.9
_C.model.patience = 3

_C.n_folds = 10
_C.num_classes = len(targets)
_C.seed = 42
_C.metric = 'LogLoss'
_C.device = "cuda" if torch.cuda.is_available() else "cpu"

def get_cfg_defaults():
    """Get a yacs CfgNode object with default values for my_project."""
    # Return a clone so that the defaults will not be altered
    # This is for the "local variable" use pattern
    #return _C.clone()
    return _C

def dump_cfg(config = get_cfg_defaults() , path = "experiment.yaml"):
    """Save a yacs CfgNode object in a yaml file in path."""
    stream = open(path, 'w')
    stream.write(config.dump())
    stream.close()

def inject_config(funct):
    """Inject a yacs CfgNode object in a function as first arg."""
    @wraps(funct)
    def function_wrapper(*args,**kwargs):
        return funct(*args,**kwargs,config=_C)  
    return function_wrapper

def dump_dict(config,path="config.yaml"):
        stream = open(path, 'w')
        yaml.dump(config,stream)
        stream.close()

c=get_cfg_defaults()

# Utils

In [ ]:
class ImageDataset(Dataset):
    @inject_config
    def __init__(self,df, transform=None,mode='train',config =CN):
        self.df = df
        self.transform = transform
        self.mode = mode
        self.dir = config.data.data_path
        self.Image_ID_col = config.data.Image_ID_col
        self.LABELS = config.data.LABELS

    def cv_reader(self,path):
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img

    def __getitem__(self, index):
        image_name = os.path.join(self.dir,f"{self.df[self.Image_ID_col][index].split('.')[0]}.jpg")
        image = self.cv_reader(image_name)
        
        if self.transform is not None:
          image = self.transform(image=image)
          image=image["image"]

        if self.mode == 'train':
            label = self.df[self.LABELS][index]
            return {'image' : torch.tensor(image,dtype=torch.float), 
                    'label' : torch.tensor(label,dtype = torch.float) }
        return {'image' : torch.tensor(image,dtype=torch.float)}
        
    def __len__(self):
        return self.df.shape[0]

In [ ]:
class SwinModel(nn.Module):
    @inject_config
    def __init__(self,model_name,config:CN):
        super().__init__()
        if 'swin' in model_name or 'visformer_small' in model_name: 
          self.model = timm.create_model(model_name, pretrained=True)
          n_features = self.model.head.in_features
          self.model.head  = nn.Linear(n_features,config.num_classes)

        elif 'cait' in model_name : 
          self.model = timm.create_model(model_name, pretrained=True)
          n_features = self.model.head.in_features
          self.model.head  = nn.Linear(n_features,config.num_classes)
        else :
          self.model = timm.create_model(model_name, pretrained=True)
          n_features = self.model.head.fc.in_features
          self.model.head.fc  = nn.Linear(n_features,config.num_classes)
    def forward(self, x):
        output = self.model(x)
        return output

In [ ]:
class AverageMeter():
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
class Trainer :
  @inject_config
  def __init__(self,config : CN) :
    self.device = config.device
    self.n_accumulate = config.model.n_accumulate
    
  def get_score(self,y_true, y_pred):
    return f1_score(y_true, np.argmax(y_pred,axis=1),average='weighted')

  def loss_fn(self,outputs,targets):
    criterion = nn.CrossEntropyLoss()
    loss = criterion(outputs,targets)
    return loss
  
  def train_fn(self,train_data_loader,model,optimizer,scheduler = None):
    from  torch.cuda import amp
    scaler = amp.GradScaler()

    model.train()
    losses = AverageMeter()
    roc_auc = AverageMeter()
    tk0 = tqdm(train_data_loader, total=len(train_data_loader))
    tot_loss = 0
    for step,d in enumerate(tk0):
      images =  d['image']
      labels =  d['label']
      #send them to device 
      images  = images.to(self.device,dtype=torch.float)
      labels  = labels.to(self.device,dtype=torch.long)

      with amp.autocast(enabled=True):
        outputs  = model(images)
        loss = self.loss_fn(outputs,labels)
      
      # Accumulates gradients after scale.
      scaler.scale(loss).backward()

      if (step+ 1) % self.n_accumulate == 0:
          scaler.step(optimizer)
          scaler.update()
          optimizer.zero_grad()

      if scheduler is not None:
        scheduler.step()

      tot_loss = tot_loss + loss.item()
      losses.update(loss.item(), labels.size(0))
      tk0.set_postfix(loss=losses.avg)
      

    loss_score = tot_loss/len(train_data_loader)
    f1_scoree = self.get_score(labels.cpu().numpy(), torch.nn.functional.softmax(outputs).cpu().detach().numpy())
    
    print(f'LR for this Epoch :: {optimizer.param_groups[0]["lr"]}')
    print("Training loss for this epoch: ", loss_score)
    print("Training F1 Score for this epoch: ", f1_scoree)
    return f1_scoree

  def eval_fn(self,valid_data_loader,model):
    model.eval()
    tot_loss = 0
    final_outputs = []
    final_targets = []
    with torch.no_grad():
      for bi,d in enumerate(valid_data_loader):
        images = d['image']
        labels = d['label']
        #send them to device 
        images = images.to(self.device,dtype=torch.float)
        labels = labels.to(self.device,dtype=torch.long)
        
        outputs  = model(images)
        loss = self.loss_fn(outputs,labels)
        tot_loss = tot_loss + loss.item()

        final_outputs.append(torch.nn.functional.softmax(outputs).cpu().detach().numpy())
        final_targets.append(labels.cpu().numpy())

      final_targets = np.concatenate(final_targets)
      final_outputs = np.concatenate(final_outputs)
      
      loss_score  = tot_loss/len(valid_data_loader)
      f1_scoree = self.get_score(final_targets,final_outputs)
      print("Validation loss for this epoch: ",loss_score)
      print('Validation F1 Score for this epoch',f1_scoree)
    return loss_score,f1_scoree , final_outputs

In [ ]:
class Augmentation :
  @inject_config
  def __init__(self,config :CN) :
    self.input_shape = config.preprocess.input_shape
    self.SEED_VAL  = config.seed
    self.crop_shape  = config.preprocess.crop_shape
    
  def seed_all(self):
        random.seed(self.SEED_VAL)
        np.random.seed(self.SEED_VAL)
        torch.manual_seed(self.SEED_VAL)
        torch.cuda.manual_seed_all(self.SEED_VAL)
        os.environ['PYTHONHASHSEED'] = str(self.SEED_VAL)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
  def train_transform(self,) :
    self.seed_all()
    train_transform = A.Compose([
                              A.OneOf([
                                    A.RandomResizedCrop (self.input_shape[0], self.input_shape[1], scale=(0.8, 1.0), ratio=(0.8, 1.0), p=0.2),
                                    A.Compose([ 
                                        A.RandomCrop(self.crop_shape[0], self.crop_shape[1],p=1.0),
                                        A.Resize(height=self.input_shape[0], width=self.input_shape[1], p=1.0)
                                        ],p=0.3),      
                                    A.Resize(height=self.input_shape[0], width=self.input_shape[1], p=0.5),
                                  ]
                                  , p=1),
                              OneOf([
                                    A.RandomFog(p=0.5),
                                    A.Blur(blur_limit=10, p=.1), 
                                     A.MotionBlur(p=0.2)
                              ], p=0.1),
                              OneOf([CLAHE(clip_limit=[2,4],p=0.5),
                                     A.Equalize(p=0.3),
                                     A.HueSaturationValue(p=0.3),
                                    ], p=0.3),
                              OneOf([
                                     IAASharpen(p=0.1),
                                     A.RandomBrightnessContrast(p=0.5),
                                     RandomContrast(p=0.5),
                                     RandomBrightness(p=0.5),
                              ], p=0.3),
                              OneOf([ 
                                  A.Transpose(p=0.5),
                                  A.VerticalFlip(0.5),
                              ], p=0.3),
                              A.HorizontalFlip(p=0.3),
                              OneOf([A.Rotate (limit=45, interpolation=1, border_mode=4,p=0.7), # change p
                                     AG.Affine(translate_percent=0.1,rotate=10,interpolation=1,mode=cv2.BORDER_REPLICATE, p=0.3) , 
                                ], p=0.3),
                              A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), p=1.0),
                              ToTensorV2(),
                              ])
    return train_transform
  
  def test_trasnform(self,Augstype='Resize') :
    if Augstype == 'Resize' :
        test_transform = A.Compose([ 
                                  A.Resize(height=self.input_shape[0], width=self.input_shape[1],p=1),
                                  A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), p=1.0),
                                  ToTensorV2(),
                                  ])
        
    return test_transform

In [ ]:
class PytorchTrainer :
  @inject_config
  def __init__(self, config:CN) :
    
    self.model_name  = config.model.name_model
    self.EPOCHS  = config.model.epochs
    self.lr  = config.model.base_lr
    self.weight_decay = config.model.weight_decay
    self.train_bs = config.model.train_bs
    self.test_bs = config.model.test_bs
    self.LABELS = config.data.LABELS
    self.NTargets = config.num_classes
    self.device   = config.device
    self.SEED_VAL  = config.seed
    self.n_splits  = config.n_folds
    self.metric = config.metric
    self.n_accumulate = config.model.n_accumulate

  def fetch_scheduler(self,optimizer):
    if _C.model.scheduler == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CFG.T_max, 
                                                   eta_min=_C.model.min_lr)
    elif _C.model.scheduler == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CFG.T_0, 
                                                             eta_min=_C.model.min_lr)
    elif _C.model.scheduler == 'ReduceLROnPlateau':
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer,
                                                   mode='min',
                                                   factor=_C.model.factor,
                                                   patience=_C.model.patience,
                                                   threshold=0.00001,
                                                   min_lr=_C.model.min_lr,)
    elif _C.model.scheduler == 'ExponentialLR':
        scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.85)
    elif _C.model.scheduler == None:
        return None
        
    return scheduler

  def get_score(self,y_true, y_pred):
    return accuracy_score(y_true, np.argmax(y_pred,axis=1))

  def seed_all(self):
        random.seed(self.SEED_VAL)
        np.random.seed(self.SEED_VAL)
        torch.manual_seed(self.SEED_VAL)
        torch.cuda.manual_seed_all(self.SEED_VAL)
        os.environ['PYTHONHASHSEED'] = str(self.SEED_VAL)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False 

  def kfold_split(self,Train,y):
        Train["folds"]=-1
        kf = StratifiedKFold(n_splits= self.n_splits,random_state=self.SEED_VAL,shuffle=True)
        for fold, (_, val_index) in enumerate(kf.split(Train,y)):
                Train.loc[val_index, "folds"] = fold
        return Train
  
  def Train_One_Fold(self,train_df , valid_df,fold):
    self.seed_all()
    trainer = Trainer()
    augs = Augmentation()

    train_transform = augs.train_transform()
    val_transform   = augs.test_trasnform()

    train_dataset = ImageDataset(train_df.reset_index(),train_transform)
    valid_dataset = ImageDataset(valid_df.reset_index(),val_transform)
    
    train_data_loader = DataLoader(dataset=train_dataset,shuffle=True,batch_size=self.train_bs//self.n_accumulate)
    valid_data_loader = DataLoader(dataset=valid_dataset,shuffle=False,batch_size=self.test_bs)
    
    if 'swin' in self.model_name or 'cait' in self.model_name or 'cait' in self.model_name: 
        model = SwinModel(model_name=self.model_name) 
        model.to(self.device)
    

    optimizer = torch.optim.Adam(model.parameters(), lr=self.lr, weight_decay=self.weight_decay)
    scheduler = self.fetch_scheduler(optimizer)
    
    best_score = np.inf
    for epoch in range(self.EPOCHS):
      print("----------------EPOCH "+str(epoch+1)+"---------------------")
      Acc_train = trainer.train_fn(train_data_loader, model, optimizer,scheduler=None)
      loss_val,Score_val,oof_ = trainer.eval_fn(valid_data_loader ,model)
      scheduler.step(loss_val)
      if loss_val<best_score:
        best_score = loss_val 
        oof_1f = np.copy(oof_) 
        torch.save(model.state_dict(),f"best_model_{fold}") 
    return oof_1f , best_score 

  def Train_K_folds(self,train):
    self.seed_all()
    oof = np.zeros((train.shape[0],self.NTargets))
    train = self.kfold_split(train,train[self.LABELS])
    
    for fold in range(self.n_splits):
      print(f"#########################  Fold {fold+1}/{self.n_splits}  #########################")
      train_df = train[train['folds']!=fold]
      valid_df = train[train['folds']==fold]
      oof_1f , best_score = self.Train_One_Fold(train_df , valid_df,fold)
      oof[valid_df.index.tolist()] = oof_1f 
      free_memory(sleep_time=0.1)
    print(f'{self.metric} OOF Score :',self.get_score(train[self.LABELS],oof))
    return oof

  def InferenceValid(self,valid_df,Augstype='Resize') :
    self.seed_all()
    print(f'Inference On {Augstype} Augs ...')
    augs = Augmentation()
    test_transform   = augs.test_trasnform(Augstype=Augstype)
    valid_dataset = ImageDataset(valid_df.reset_index(drop=True),test_transform,mode='test')
    test_data_loader = DataLoader(dataset=valid_dataset,shuffle=False,batch_size=self.test_bs)
    if 'swin' in self.model_name or 'cait' in self.model_name or 'cait' in self.model_name: 
        best_model = SwinModel(model_name=self.model_name) 
        best_model.to(self.device)
    best_model.load_state_dict(torch.load(f'best_model_{fold}'))
    best_model.to(self.device)
    best_model.eval()
    
    final_outputs = []
    with torch.no_grad() :
        tk0 = tqdm(test_data_loader, total=len(test_data_loader))
        for bi,d in enumerate(tk0):
          images = d['image']
          #send them to device 
          images = images.to(self.device,dtype=torch.float)
          outputs  = best_model(images)
          final_outputs.append(torch.nn.functional.softmax(outputs).cpu().detach().numpy())

    final_outputs = np.concatenate(final_outputs)
    Final_Prediction   = np.argmax(final_outputs,axis=1)
    return  final_outputs , Final_Prediction 

  def Inference(self,test,Augstype='Resize') :
    augs = Augmentation()
    test_transform   = augs.test_trasnform(Augstype=Augstype)

    test_dataset = ImageDataset(test.reset_index(drop=True),test_transform,mode='test')
    test_data_loader = DataLoader(dataset=test_dataset,shuffle=False,batch_size=self.test_bs)

    all_folds = []
    for fold in range(self.n_splits):
      print(f"#########################  Fold {fold+1}/{self.n_splits}  #########################")
      if 'swin' in self.model_name or 'cait' in self.model_name or 'cait' in self.model_name: 
        best_model = SwinModel(model_name=self.model_name) 
        best_model.to(self.device)
        
      best_model.load_state_dict(torch.load(f'best_model_{fold}'))
      best_model.to(self.device)
      best_model.eval()
      final_outputs = []
      with torch.no_grad() :
        tk0 = tqdm(test_data_loader, total=len(test_data_loader))
        for bi,d in enumerate(tk0):
          images = d['image']
          #send them to device 
          images = images.to(self.device,dtype=torch.float)
          outputs  = best_model(images)
          final_outputs.append(F.softmax(outputs, dim=1).data.squeeze().cpu().detach().numpy())

      final_outputs = np.concatenate(final_outputs)
      all_folds.append(final_outputs)
    Final_Prediction_f = np.mean(all_folds,axis=0)
    Final_Prediction_gmean = gmean(all_folds)
    Final_Prediction   = np.argmax(Final_Prediction_f,axis=1)
    return  Final_Prediction_f, Final_Prediction_gmean, Final_Prediction

In [ ]:
AssazzinTrainer = PytorchTrainer()

# **Training**

In [ ]:
import gc ; gc.collect()

100

In [ ]:
import gc
import time
def free_memory(sleep_time=0.1) :
    """ Black magic function to free torch memory and some jupyter whims """
    gc.collect()
    torch.cuda.synchronize()
    gc.collect()
    torch.cuda.empty_cache()
    time.sleep(sleep_time)

free_memory(sleep_time=0.1)

In [ ]:
OOF = AssazzinTrainer.Train_K_folds(train=Train)

# Predicting

In [ ]:
free_memory(sleep_time=0.1)

In [ ]:
Test = Test[~Test.Image_id.str.contains('_rgn')].reset_index(drop=True) # Just the RGB images

In [ ]:
Final_Prediction_f1, Final_Prediction_gmean_f1, _ = AssazzinTrainer.Inference(test=Test,Augstype ='Resize')

In [ ]:
Final_Prediction = Final_Prediction_gmean_f1 

In [ ]:
np.save(f'AssazzinBaseline_{c.model.name_model}_Preds_gmean_10Folds.npy',Final_Prediction)

In [ ]:
import gc ; gc.collect()

455